In [1]:
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import linregress
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
import gzip
import pickle
import pandasql
import decimal

In [2]:
path = 'zippedData/im.db'
conn = sqlite3.connect(path)

cur = conn.cursor()
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")

table_names = cur.fetchall() # Fetch the result and store it in table_names
table_names

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [3]:
cur.execute("""SELECT * FROM movie_basics;""")

In [4]:
df = pd.read_sql(
    """
    SELECT *
    FROM sqlite_master
    """, conn)
df[df['type'] == 'table']

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [5]:
movie_basics=pd.read_sql("""SELECT * FROM movie_basics ;""", conn)
movie_basics = movie_basics.rename(columns={'primary_title': 'movie'})
movie_basics

,movie_id,movie,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [7]:
directors=pd.read_sql(
"""
SELECT *
FROM directors
;
""", conn)
directors

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502
...,...,...
291169,tt8999974,nm10122357
291170,tt9001390,nm6711477
291171,tt9001494,nm10123242
291172,tt9001494,nm10123248


In [8]:
known_for=pd.read_sql(
"""
SELECT *
FROM known_for
;
""", conn)
known_for

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534
...,...,...
1638255,nm9990690,tt9090932
1638256,nm9990690,tt8737130
1638257,nm9991320,tt8734436
1638258,nm9991320,tt9615610


In [9]:
movie_akas=pd.read_sql(
"""
SELECT *
FROM movie_akas
;
""", conn)
movie_akas

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [10]:
movie_ratings=pd.read_sql(
"""
SELECT *
FROM movie_ratings
;
""", conn)
movie_ratings

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [11]:
persons=pd.read_sql(
"""
SELECT *
FROM persons
;
""", conn)
persons

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,actress
606644,nm9990690,Joo Yeon So,NaN,NaN,actress
606645,nm9991320,Madeline Smith,NaN,NaN,actress
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer


In [12]:
principals=pd.read_sql(
"""
SELECT *
FROM principals
;
""", conn)
principals

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"
...,...,...,...,...,...,...
1028181,tt9692684,1,nm0186469,actor,None,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,None,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,None,None
1028184,tt9692684,4,nm6009913,writer,writer,None


In [13]:
writers=pd.read_sql(
"""
SELECT *
FROM writers
;
""", conn)
writers

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087
...,...,...
255868,tt8999892,nm10122246
255869,tt8999974,nm10122357
255870,tt9001390,nm6711477
255871,tt9004986,nm4993825


In [14]:
DBs_Master = movie_basics.merge(known_for, on='movie_id', how='inner')
DBs_Master = DBs_Master.merge(movie_ratings, on='movie_id', how='inner')
DBs_Master

,movie_id,movie,original_title,start_year,runtime_minutes,genres,person_id,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",nm1194313,7.0,77
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",nm1391276,7.0,77
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,nm0462648,6.9,4517
3,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,nm0613657,6.9,4517
4,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,nm1152466,6.9,4517
...,...,...,...,...,...,...,...,...,...
526723,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",nm4394529,8.7,136
526724,tt9914642,Albatross,Albatross,2017,NaN,Documentary,nm3980661,8.5,8
526725,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,nm3827360,6.6,5
526726,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,nm1434828,6.6,5


In [37]:
DBs_PERSON = writers.merge(persons, on='person_id', how='inner', left_index=True, right_index=True)
DBs_PERSON = DBs_PERSON.merge(principals, on='person_id', how='inner', left_index=True, right_index=True)
DBs_PERSON = DBs_PERSON.merge(known_for, on='person_id', how='inner', left_index=True, right_index=True)
DBs_PERSON = DBs_PERSON.merge(directors, on='person_id', how='inner', left_index=True, right_index=True)
DBs_PERSON

,movie_id_x,person_id,primary_name,birth_year,death_year,primary_profession,movie_id_y,ordering,category,job,characters,movie_id_x,movie_id_y
0,tt0285252,nm0899854,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer",tt0111414,1,actor,None,"[""The Man""]",tt0837562,tt0285252
1,tt0438973,nm0175726,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department",tt0111414,2,director,None,None,tt2398241,tt0462036
2,tt0438973,nm1802864,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer",tt0111414,3,producer,producer,None,tt0844471,tt0835418
3,tt0462036,nm1940585,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department",tt0323808,10,editor,None,None,tt0118553,tt0835418
4,tt0835418,nm0310087,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator",tt0323808,1,actress,None,"[""Beth Boothby""]",tt0896534,tt0878654
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255868,tt8999892,nm10122246,Dimitris Vlahopanos,NaN,NaN,"actor,writer,producer",tt0906778,3,actor,None,"[""Abraham Jonker""]",tt0224581,tt5910350
255869,tt8999974,nm10122357,Peter Ahr,NaN,NaN,miscellaneous,tt0906778,4,actor,None,"[""Uys Krige""]",tt0159888,tt5910736
255870,tt9001390,nm6711477,Cedric Cirotteau,1980.0,NaN,"actor,producer,stunts",tt0906778,5,director,None,None,tt0226209,tt5914618
255871,tt9004986,nm4993825,Terry Allen Fraser,NaN,NaN,"writer,producer",tt0906778,6,writer,None,None,tt0140806,tt5914618


In [38]:
DBs_PERSON = DBs_PERSON.drop(columns=['movie_id_y', 'movie_id_y'])
DBs_PERSON

,movie_id_x,person_id,primary_name,birth_year,death_year,primary_profession,ordering,category,job,characters,movie_id_x
0,tt0285252,nm0899854,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer",1,actor,None,"[""The Man""]",tt0837562
1,tt0438973,nm0175726,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department",2,director,None,None,tt2398241
2,tt0438973,nm1802864,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer",3,producer,producer,None,tt0844471
3,tt0462036,nm1940585,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department",10,editor,None,None,tt0118553
4,tt0835418,nm0310087,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator",1,actress,None,"[""Beth Boothby""]",tt0896534
...,...,...,...,...,...,...,...,...,...,...,...
255868,tt8999892,nm10122246,Dimitris Vlahopanos,NaN,NaN,"actor,writer,producer",3,actor,None,"[""Abraham Jonker""]",tt0224581
255869,tt8999974,nm10122357,Peter Ahr,NaN,NaN,miscellaneous,4,actor,None,"[""Uys Krige""]",tt0159888
255870,tt9001390,nm6711477,Cedric Cirotteau,1980.0,NaN,"actor,producer,stunts",5,director,None,None,tt0226209
255871,tt9004986,nm4993825,Terry Allen Fraser,NaN,NaN,"writer,producer",6,writer,None,None,tt0140806


In [39]:
SQL_master = pd.merge(DBs_PERSON, DBs_Master, on='person_id', how='inner', left_index=True, right_index=True)
SQL_master

,movie_id_x,person_id,primary_name,birth_year,death_year,primary_profession,ordering,category,job,characters,movie_id_x,movie_id,movie,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0285252,nm0899854,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer",1,actor,None,"[""The Man""]",tt0837562,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0438973,nm0175726,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department",2,director,None,None,tt2398241,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
2,tt0438973,nm1802864,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer",3,producer,producer,None,tt0844471,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,tt0462036,nm1940585,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department",10,editor,None,None,tt0118553,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
4,tt0835418,nm0310087,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator",1,actress,None,"[""Beth Boothby""]",tt0896534,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255868,tt8999892,nm10122246,Dimitris Vlahopanos,NaN,NaN,"actor,writer,producer",3,actor,None,"[""Abraham Jonker""]",tt0224581,tt3186318,Hellion,Hellion,2014,99.0,"Drama,Thriller",6.1,5086
255869,tt8999974,nm10122357,Peter Ahr,NaN,NaN,miscellaneous,4,actor,None,"[""Uys Krige""]",tt0159888,tt3186318,Hellion,Hellion,2014,99.0,"Drama,Thriller",6.1,5086
255870,tt9001390,nm6711477,Cedric Cirotteau,1980.0,NaN,"actor,producer,stunts",5,director,None,None,tt0226209,tt3186318,Hellion,Hellion,2014,99.0,"Drama,Thriller",6.1,5086
255871,tt9004986,nm4993825,Terry Allen Fraser,NaN,NaN,"writer,producer",6,writer,None,None,tt0140806,tt3186318,Hellion,Hellion,2014,99.0,"Drama,Thriller",6.1,5086


# # DATAFRAMES BELOW

In [15]:
movie_gross = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
movie_info = pd.read_table('zippedData/rt.movie_info.tsv.gz')
rt_reviews = pd.read_table('zippedData/rt.reviews.tsv.gz', encoding='unicode_escape')
tmdb_movies = pd.read_csv('zippedData/tmdb.movies.csv.gz')
movie_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

In [16]:
with gzip.open('zippedData/bom.movie_gross.csv.gz', 'rb') as f:
    movie_gross = pd.read_csv(f)
for col in movie_gross.select_dtypes(include=['object']):
    movie_gross[col] = movie_gross[col].str.lower()

movie_gross = movie_gross.rename(columns={'title': 'movie'})
    
movie_grossCLEAN = movie_gross.applymap(lambda x: str(x).replace(',', '').replace('$', ''))
movie_grossCLEAN

,movie,studio,domestic_gross,foreign_gross,year
0,toy story 3,bv,415000000.0,652000000,2010
1,alice in wonderland (2010),bv,334200000.0,691300000,2010
2,harry potter and the deathly hallows part 1,wb,296000000.0,664300000,2010
3,inception,wb,292600000.0,535700000,2010
4,shrek forever after,p/dw,238700000.0,513900000,2010
...,...,...,...,...,...
3382,the quake,magn.,6200.0,nan,2018
3383,edward ii (2018 re-release),fm,4800.0,nan,2018
3384,el pacto,sony,2500.0,nan,2018
3385,the swan,synergetic,2400.0,nan,2018


In [17]:
with gzip.open('zippedData/rt.movie_info.tsv.gz', 'rb') as f:
    movie_info = pd.read_csv(f, delimiter='\t', encoding='unicode_escape')
for col in movie_info.select_dtypes(include=['object']):
    movie_info[col] = movie_info[col].str.lower()
    
movie_infoCLEAN = movie_info.applymap(lambda x: str(x).replace(',', '').replace('$', ''))
movie_infoCLEAN

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,this gritty fast-paced and innovative police d...,r,action and adventure|classics|drama,william friedkin,ernest tidyman,oct 9 1971,sep 25 2001,nan,nan,104 minutes,nan
1,3,new york city not-too-distant-future: eric pac...,r,drama|science fiction and fantasy,david cronenberg,david cronenberg|don delillo,aug 17 2012,jan 1 2013,,600000,108 minutes,entertainment one
2,5,illeana douglas delivers a superb performance ...,r,drama|musical and performing arts,allison anders,allison anders,sep 13 1996,apr 18 2000,nan,nan,116 minutes,nan
3,6,michael douglas runs afoul of a treacherous su...,r,drama|mystery and suspense,barry levinson,paul attanasio|michael crichton,dec 9 1994,aug 27 1997,nan,nan,128 minutes,nan
4,7,nan,nr,drama|romance,rodney bennett,giles cooper,nan,nan,nan,nan,200 minutes,nan
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,forget terrorists or hijackers -- there's a ha...,r,action and adventure|horror|mystery and suspense,nan,nan,aug 18 2006,jan 2 2007,,33886034,106 minutes,new line cinema
1556,1997,the popular saturday night live sketch was exp...,pg,comedy|science fiction and fantasy,steve barron,terry turner|tom davis|dan aykroyd|bonnie turner,jul 23 1993,apr 17 2001,nan,nan,88 minutes,paramount vantage
1557,1998,based on a novel by richard powell when the lo...,g,classics|comedy|drama|musical and performing arts,gordon douglas,nan,jan 1 1962,may 11 2004,nan,nan,111 minutes,nan
1558,1999,the sandlot is a coming-of-age story about a g...,pg,comedy|drama|kids and family|sports and fitness,david mickey evans,david mickey evans|robert gunter,apr 1 1993,jan 29 2002,nan,nan,101 minutes,nan


In [35]:
ID_1 = pd.merge(SQL_master, movie_infoCLEAN, on='movie', how='inner')
ID_1

KeyError: 'movie'

In [18]:
with gzip.open('zippedData/rt.reviews.tsv.gz', 'rb') as f:
    rt_reviews = pd.read_csv(f, delimiter='\t', encoding='unicode_escape')
for col in rt_reviews.select_dtypes(include=['object']):
    rt_reviews[col] = rt_reviews[col].str.lower()
    rt_reviewsCLEAN = rt_reviews.applymap(lambda x: str(x).replace(',', '').replace('$', ''))
rt_reviewsCLEAN

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,a distinctly gallows take on contemporary fina...,3/5,fresh,pj nabarro,0,patrick nabarro,november 10 2018
1,3,it's an allegory in search of a meaning that n...,nan,rotten,annalee newitz,0,io9.com,may 23 2018
2,3,... life lived in a bubble in financial dealin...,nan,fresh,sean axmaker,0,stream on demand,january 4 2018
3,3,continuing along a line introduced in last yea...,nan,fresh,daniel kasman,0,mubi,november 16 2017
4,3,... a perverse twist on neorealism...,nan,fresh,nan,0,cinema scope,october 12 2017
...,...,...,...,...,...,...,...,...
54427,2000,the real charm of this trifle is the deadpan c...,nan,fresh,laura sinagra,1,village voice,september 24 2002
54428,2000,nan,1/5,rotten,michael szymanski,0,zap2it.com,september 21 2005
54429,2000,nan,2/5,rotten,emanuel levy,0,emanuellevy.com,july 17 2005
54430,2000,nan,2.5/5,rotten,christopher null,0,filmcritic.com,september 7 2003


In [42]:
with gzip.open('zippedData/tmdb.movies.csv.gz', 'rb') as f:
    tmdb_movies = pd.read_csv(f)

for col in tmdb_movies.select_dtypes(include=['object']):
    tmdb_movies[col] = tmdb_movies[col].str.lower()

tmdb_movies = tmdb_movies.rename(columns={'original_title': 'movie'})
tmdb_movies['genre_ids'] = tmdb_movies['genre_ids'].str.replace('[', '').str.replace(']', '')

tmdb_moviesCLEAN = tmdb_movies.applymap(lambda x: str(x).replace(',', '').replace('$', ''))

# Define a dictionary to map genre ID numbers to genre names
genre_id_mapping = {
    '28': 'action',
    '12': 'adventure',
    '16': 'animation',
    '35': 'comedy',
    '80': 'crime',
    '99': 'documentary',
    '18': 'drama',
    '10751': 'family',
    '14': 'fantasy',
    '36': 'history',
    '27': 'horror',
    '10402': 'music',
    '9648': 'mystery',
    '878': 'science fiction',
    '10770': 'tv movie',
    '53': 'thriller',
    '10752': 'war',
    '37': 'western'
}

# Replace genre ID numbers with genre names in the 'genre_ids' column
for genre_id, genre_name in genre_id_mapping.items():
    tmdb_moviesCLEAN['genre_ids'] = tmdb_moviesCLEAN['genre_ids'].str.replace(genre_id, genre_name)
tmdb_moviesCLEAN.rename(columns={'movieNAME': 'movie'}, inplace=True)

tmdb_moviesCLEAN = tmdb_moviesCLEAN.rename(columns={'movie': 'original_title'})

tmdb_moviesCLEAN

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,adventure fantasy family,12444,en,harry potter and the deathly hallows: part 1,33.533,2010-11-19,harry potter and the deathly hallows: part 1,7.7,10788
1,1,fantasy adventure animation family,10191,en,how to train your dragon,28.734,2010-03-26,how to train your dragon,7.7,7610
2,2,adventure action science fiction,10138,en,iron man 2,28.515,2010-05-07,iron man 2,6.8,12368
3,3,animation comedy family,862,en,toy story,28.005,1995-11-22,toy story,7.9,10174
4,4,action science fiction adventure,27205,en,inception,27.92,2010-07-16,inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,horror drama,488143,en,laboratory conditions,0.6,2018-10-13,laboratory conditions,0.0,1
26513,26513,drama thriller,485975,en,_exhibit_84xxx_,0.6,2018-05-01,_exhibit_84xxx_,0.0,1
26514,26514,fantasy action adventure,381231,en,the last one,0.6,2018-10-01,the last one,0.0,1
26515,26515,family adventure action,366854,en,trailer made,0.6,2018-06-22,trailer made,0.0,1


In [20]:
with gzip.open('zippedData/tn.movie_budgets.csv.gz', 'rb') as f:
    movie_budgets = pd.read_csv(f)
# lowercase all strings
for col in movie_budgets.select_dtypes(include=['object']):
    movie_budgets[col] = movie_budgets[col].str.lower()
# remove all ',' and '$' delimiters 
    movie_budgetsCLEAN = movie_budgets.applymap(lambda x: str(x).replace(',', '').replace('$', ''))
movie_budgetsCLEAN

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,dec 18 2009,avatar,425000000,760507625,2776345279
1,2,may 20 2011,pirates of the caribbean: on stranger tides,410600000,241063875,1045663875
2,3,jun 7 2019,dark phoenix,350000000,42762350,149762350
3,4,may 1 2015,avengers: age of ultron,330600000,459005868,1403013963
4,5,dec 15 2017,star wars ep. viii: the last jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5777,78,dec 31 2018,red 11,7000,0,0
5778,79,apr 2 1999,following,6000,48482,240495
5779,80,jul 13 2005,return to the land of wonders,5000,1338,1338
5780,81,sep 29 2015,a plague so pleasant,1400,0,0


In [ ]:
merged_df1 = pd.merge(movie_budgetsCLEAN, tmdb_moviesCLEAN, on='movie', how='inner')
merged_df1

In [26]:
MASTER1 = pd.merge(movie_budgetsCLEAN, tmdb_moviesCLEAN, on='movie', how='inner')
MASTER1 = pd.merge(MASTER1, movie_grossCLEAN, on='movie', how='inner')
MASTER1

,id_x,release_date_x,movie,production_budget,domestic_gross_x,worldwide_gross,Unnamed: 0,genre_ids,id_y,original_language,popularity,release_date_y,title,vote_average,vote_count,studio,domestic_gross_y,foreign_gross,year
0,2,may 20 2011,pirates of the caribbean: on stranger tides,410600000,241063875,1045663875,2470,adventure action fantasy,1865,en,30.579,2011-05-20,pirates of the caribbean: on stranger tides,6.4,8571,bv,241100000.0,804600000,2011
1,4,may 1 2015,avengers: age of ultron,330600000,459005868,1403013963,14169,action adventure science fiction,99861,en,44.383,2015-05-01,avengers: age of ultron,7.3,13457,bv,459000000.0,946400000,2015
2,7,apr 27 2018,avengers: infinity war,300000000,678815482,2048134200,23811,adventure action fantasy,299536,en,80.773,2018-04-27,avengers: infinity war,8.3,13948,bv,678800000.0,1369.5,2018
3,9,nov 17 2017,justice league,300000000,229024295,655945209,20623,action adventure fantasy science fiction,141052,en,34.953,2017-11-17,justice league,6.2,7510,wb,229000000.0,428900000,2017
4,9,nov 17 2017,justice league,300000000,229024295,655945209,23830,action adventure fantasy science fiction,141052,en,34.953,2017-11-17,justice league,6.2,7510,wb,229000000.0,428900000,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,68,apr 27 2012,sound of my voice,135000,408015,429448,2815,science fiction drama mystery thriller,86812,en,7.212000000000001,2011-01-22,sound of my voice,6.4,169,foxs,408000.0,nan,2012
1423,73,jun 15 2012,your sister's sister,120000,1597486,3090593,5468,drama comedy,84355,en,7.115,2012-06-14,your sister's sister,6.6,192,ifc,1600000.0,nan,2012
1424,80,jul 10 2015,the gallows,100000,22764410,41656474,14362,horror thriller,299245,en,9.166,2015-07-10,the gallows,4.8,591,wb (nl),22800000.0,20200000,2015
1425,86,jul 7 2017,a ghost story,100000,1594798,2769782,20776,drama fantasy 10749,428449,en,11.886,2017-07-07,a ghost story,7.0,906,a24,1600000.0,nan,2017


In [ ]:
MASTER1 DBs_Master

In [29]:
MASTER_MOVIE= pd.merge(MASTER1, DBs_Master, on='movie', how='outer')
MASTER_MOVIE.columns

Index(['id_x', 'release_date_x', 'movie', 'production_budget',
       'domestic_gross_x', 'worldwide_gross', 'Unnamed: 0', 'genre_ids',
       'id_y', 'original_language', 'popularity', 'release_date_y', 'title',
       'vote_average', 'vote_count', 'studio', 'domestic_gross_y',
       'foreign_gross', 'year', 'movie_id', 'original_title', 'start_year',
       'runtime_minutes', 'genres', 'person_id', 'averagerating', 'numvotes'],
      dtype='object')

In [43]:
movie_id_mapping = (tmdb_moviesCLEAN.merge(movie_basics, left_on='original_title', right_on='original_title')
          .reindex(columns=['genre_ids', 'genres','movie_id', 'id', 'original_language', 'original_title', 'popularity', 'release_date', 'title', 'vote_average', 'vote_count']))
movie_id_mapping

,genre_ids,genres,movie_id,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,drama thriller,"Drama,Thriller",tt0798817,44982,en,13,8.007,2010-03-12,13,5.7,392
1,drama thriller,"Comedy,Horror",tt3848510,44982,en,13,8.007,2010-03-12,13,5.7,392
2,adventure,Adventure,tt1534084,52349,en,600 kilos d'or pur,2.634,2010-11-11,in gold we trust,5.2,22
3,comedy drama,"Comedy,Drama,Romance",tt1306980,40807,en,50/50,12.450999999999999,2011-09-30,50/50,7.1,2111
4,comedy drama,Documentary,tt1734060,40807,en,50/50,12.450999999999999,2011-09-30,50/50,7.1,2111
...,...,...,...,...,...,...,...,...,...,...,...
105,horror mystery thriller,"Comedy,Drama",tt2379653,477577,en,3,1.8159999999999998,2018-08-07,3,6.0,2
106,horror mystery thriller,"Horror,Mystery,Thriller",tt5618160,477577,en,3,1.8159999999999998,2018-08-07,3,6.0,2
107,horror mystery thriller,Drama,tt5785598,477577,en,3,1.8159999999999998,2018-08-07,3,6.0,2
108,horror mystery thriller,Documentary,tt8558372,477577,en,3,1.8159999999999998,2018-08-07,3,6.0,2
